![zebrains_logo](../../images/logo-zebrains-dark.svg)
## Кафедра «Интеллектуальных технологий в информационных системах»

# Лабораторная работа: DVC

В данной лабораторной работе вы научитесь пользоваться:
* **DVC** (Data Version Control) - инструмент версионирования данных для Машинного обучения

# DVC
<img src="../../images/dvc_logo.png" alt="drawing" width="200"/>

Данная лабораторная предполагает самостоятельное изучение документации DVC. Она прекрасно написана и хорошо структурирована. Пожалуйста, не ленитесь ее читать. Если вы будете заниматься разработкой - когда-нибудь придется писать уже свою документацию.

Если вы не уверены в своих знаниях английского и не понимаете некоторых слов - все равно читайте. Не переводите страницы целиком. Только те моменты, которые совсем не понятны. 

У вас обязательно все получится, если вы уделите этому достаточно времени.

Кратко о DVC: https://dvc.org/doc/user-guide/what-is-dvc

DVC - это система контроля данных для проектов связанных с машинным обучением.
Она хранит данные и код отдельно, в отличие от Git LFS. Это позволяет быстрее переключаться между различными конфигурациями данных и легче проводить эксперименты. 

<img src="../../images/dvc1.png" alt="drawing" width="600"/>
<img src="../../images/dvc2.png" alt="drawing" width="600"/>

Как видите, код и данные мы можем хранить отдельно. Даже на разных серверах. А когда захотим создать конфигурацию из кода+данных+параметров - мы просто указываем конкретные значения.  
Это очень удобно для ML - проектов, где нам нужно запускать множество экспериментов с разными конфигурациями.


А теперь сравните это со схемой Git LFS. Где код связан с данными. [тык](#git_lfs_cell)

С одной стороны кажется, что это очень удобно. Но это налагает дополнительные обязательства на разработчика. Придется следить, что бы разные версии данных и кода были совместимы. 

К счастью, в DVC предусмотрен этот момент. И вас предупредят, когда вы, например, закомитили файлы в Git, но забыли закомитить изменения в DVC. Этот механизм называется Хук (*hook*) - пользовательский скрипт, запускающийся, когда вы исполняете определенные команды. 

Хуки в DVC уже написаны за вас. Что бы понимать, как работают процессы под капотом - можете ознакомиться с этой страницей в документации: https://dvc.org/doc/command-reference/install

## 0. Установка DVC

Установите библиотеки (и dvc в частности) из файла с зависимостями. Рекомендуется использование окружения conda.

## 1. Инициализация

Перейдите в папку с проектом и используйте команду https://dvc.org/doc/command-reference/init
``` bash
dvc init
```

## 2. Закомитьте изменения

Git автоматически добавит файлы созданные предыдущей командой. 
Убедитесь в этом запустив 
``` bash
git status
```

Вы должны увидеть что-то вроде 
``` bash
Changes to be committed:
        new file:   .dvc/.gitignore
        new file:   .dvc/config
        ...
```

Если что-то пошло не так - мы всегда можем сделать это вручную через
``` bash
git add . # добавим все файлы (кроме тех, что прописаны в .gitignore, разумеется)
```

Наконец, закомитьте изменения
``` bash
git commit -m "Initialize DVC"
```

## 3. Добавьте данные в DVC

Добавить файлы\папки с данными в DVC (по аналогии, как вы это делали с Git)
https://dvc.org/doc/command-reference/add

``` bash
dvc add data
```

## 4. Разбейте код на стадии

*P.S. Перед непосредственным выполнением этого этапа прочтите лабу до конца. Что бы видеть общую картину.*

Представьте, что данные в вашем проекте - это материал, который должен пройти по конвейеру (pipeline).
Каждый кусок данных обрабатывается в файле - это стадия\задача\этап производства. 

Тут не нужно строить 
сложные ООП - конструкции. Посмотрите пример проекта на DVC. Постарайтесь сделать похожую архитектуру и структуру поекта (но вы всегда вольны придумать что-то своё, если, другое решение кажется вам лучше).

https://github.com/iterative/example-get-started/tree/master/src

Еще вам могут пригодиться следующие страницы:
https://dvc.org/doc/start/data-pipelines

https://dvc.org/doc/user-guide/project-structure/pipelines-files

Пример пайплайна:
![](../../images/ml-pipeline-1.png)

## 5. Соберите ваши стадии в пайплайн
Пайплайны в DVC представляют из себя файл *dvc.yaml* в котором записана последовательность стадий обработки данных.
Пример файла:

**dvc.yaml**
``` yaml
stages: # Тело пайплайна
  
  feature_extraction: # Название стадии. Из названия можно понять, что тут идет извлечение полезных параметров из данных.
    cmd: python src/featurization.py # cmd - команда в консоли, которая будет исполнена на текущей стадии
    deps: # deps - это dependencies, т.е. зависимости. Те файлы, которые будут использованы на этой стадии. 
    - data/iris.csv # файл с данными
    - src/featurization.py # файл, который будет исполняться. Мы включили его сюда на случай, если вы перепишите код, а данные не поменяются. Тогда DVC поймет, что среда изменилась.
    outs: # outs - все, что мы получим на выходе из стадии
    - data/iris_featurized.csv # файл с данными, который src/featurization.py создаст по окончанию исполнения стадии.
  
  split_dataset: # Стадия разбивки данных на тренировочную и тестовую выборку. Соответственно, на train мы будем тренировать алгоритм. А предсказания нашей модели на test покажет насколько хороший у нас получился алгоритм и как он решает поставленную задачу.
    cmd: python src/split_dataset.py
    params:
    - test_size # параметр из файла params.yaml. Говорит в каких пропорциях мы разобъем большой датасет. Например, в train пойдет 80% всех данных, а в test 20%. Например, если мы поймем, что нашему алгоритму критически не хвататет данных - можно попробовать увеличить соотношение до 85% - 15% и т.д и т.п.
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    outs: # две выборки\датасета на выходе.
    - data/test.csv
    - data/train.csv
  
  train:
    cmd: python src/train.py
    params:
    - epochs # какой-нибудь коэфициент, который мы можем менять в разных экспериментах. Например epochs - эпохи. Так в обучении нейросетей обозначается количество иттераций алогритма.
    deps:
    - data/train.csv
    - src/train.py
    outs:
    - data/model.joblib # на выходе трейна получаем модель - натренированный алгоритм
 
  evaluate: # стадия тестирования алгоритма
    cmd: python src/evaluate.py
    deps:
    - data/model.joblib
    - data/test.csv
    - src/evaluate.py
    - src/train.py
    metrics: # в метриках находятся результаты эксперимента. Например % правильно угаданых данных. Или графики тренировок, анализа и т.д. и т.п.
    - data/eval.txt
```

Вам нужно сделать похожий файл со своими стадиями. Т.е. расписать что на входе, что на выходе. Какие ресурсы используете. 

## 6. Определите ваши параметры
Пайплайны в DVC представляют из себя файл *params.yaml* в котором записана последовательность стадий обработки данных.
Пример файла:

**params.yaml**

```yaml
split_dataset:
  test_size: 0.80

train:
  epochs: 100
```

## 7. Получите параметры из конфигурационного файла в коде
Пример с сайта DVC
```python
import yaml

with open("params.yaml", 'r') as fd:
    params = yaml.safe_load(fd)

lr = params['lr']
epochs = params['train']['epochs']
layers = params['train']['layers']
```

## 8. Обработайте с помощью пайплайна данные.
Для этого вам потребуется команда repro https://dvc.org/doc/command-reference/repro
``` bash
dvc repro
```
Вы можете запускать как отдельные стадии, так и весь пайплайн целиком. Можно запускать стадии с разными параметрами, напрямую переназначая параметры. Подробнее в документации.

При каждом запуске стадии dvc проверит данные, что бы вы не тратили время на запуск одних и тех же команд. Но если вы уверены, что вам нужно пропустить данные через ваш пайплайн или стадию - добавьть флаг *--force* или просто *-f*.

### Отчет

Прикрепите следующие скриншоты прямо внутри ноутбука.

Результаты команд
``` bash
cat dvc.yaml
cat params.yaml
dvc dag
dvc list . --dvc-only
dvc repro -f
```

In [ ]:
uploader = FileUpload(accept = 'image/*', multiple = True)
display(uploader)

In [ ]:
display_uploaded_screenshots(uploader.data)